In [1]:
import os
from pathlib import Path
from openai import OpenAI

client = OpenAI(
    api_key=''  
)


In [2]:
xuhao=1
dataset='Yelp'
stage='item'

In [ ]:

# test.jsonl 是一个本地示例文件，purpose必须是batch
file_object = client.files.create(file=Path(f"{dataset}_{stage}request_{xuhao}.jsonl"), purpose="batch")

print(file_object.model_dump_json())
print(file_object.id)  # 打印文件id
file_object_id=file_object.id

In [ ]:
batch = client.batches.create(
    input_file_id=file_object_id,  # 上传文件返回的 id
    endpoint="/v1/chat/completions",  # 大语言模型固定填写，/v1/chat/completions
    completion_window="24h"  # 当前只支持24h，24小时未运行完会超时
)
print(batch)
print(batch.id)  # 打印Batch任务的id
batch_id=batch.id

In [ ]:
#查询
batch = client.batches.retrieve('')  # 将batch_id替换为Batch任务的id
print(batch)
print(batch.error_file_id)
print(batch.output_file_id)  # 打印输出文件id
error_file_id=batch.error_file_id
output_file_id=batch.output_file_id

In [ ]:
content = client.files.content(file_id=error_file_id)
# 打印结果文件内容
print(content.text)
# 保存结果文件至本地
content.write_to_file(f"{dataset}_{stage}resulterror_{xuhao}.jsonl")

In [ ]:
content = client.files.content(file_id=output_file_id)
# 打印结果文件内容
print(content.text)
# 保存结果文件至本地
content.write_to_file(f"{dataset}_{stage}result_{xuhao}.jsonl")

In [ ]:
import json

# 定义函数将返回的jsonl数据解析成json列表
def parse_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = []
        for line in file:
            data.append(json.loads(line))
    return data

# 提取 custom_id 和 message 中的 content
def extract_custom_id_and_content(parsed_data, extracted_data):
    for item in parsed_data:
        custom_id = item.get("custom_id")
        content = item["response"]["body"]["choices"][0]["message"]["content"]
        extracted_data[custom_id] = content

# 保存提取的数据到一个json文件
def save_to_json(data, output_path):
    with open(output_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
def delete_files(file_paths):
    for file_path in file_paths:
        try:
            os.remove(file_path)
        except FileNotFoundError:
            print(f"文件 {file_path} 未找到，跳过删除。")

# 多个 JSONL 文件路径
jsonl_files = [f'{dataset}_{stage}result_1.jsonl']

# jsonl_files = ['result_1.jsonl']
# 合并所有文件中的数据
all_extracted_data = {}

# 解析每个 JSONL 文件并提取数据
for jsonl_file in jsonl_files:
    parsed_data = parse_jsonl(jsonl_file)
    extract_custom_id_and_content(parsed_data, all_extracted_data)
if stage =='item':
# 保存所有提取的数据到一个最终的 JSON 文件
    with open(f'input/caption/{dataset}_caption.json', 'r') as file:
        name = json.load(file)
    for k,v in all_extracted_data.items():
        all_extracted_data[k]=f'The item is {name[k]}, and its feature is :\n{v}'.replace('\n\n','\n')
    output_file_path = f'{dataset}_feature.json'
    save_to_json(all_extracted_data, output_file_path)
    # 删除已处理的 JSONL 文件
    # delete_files(jsonl_files)
    print(f"合并的数据已保存到 {output_file_path}")
elif stage =='user':
    output_file_path = f'{dataset}_user.json'
    save_to_json(all_extracted_data, output_file_path)
    # 删除已处理的 JSONL 文件
    # delete_files(jsonl_files)
    print(f"合并的数据已保存到 {output_file_path}")